In [2]:
from collections import defaultdict
from itertools import product
import pandas as pd
import numpy as np

In [3]:
def createDict(filename):
    benchfile = open(filename)
    bench = benchfile.read().splitlines()
    bench = [line for line in bench if '#' not in line and '' != line]
    inputs = [line for line in bench if 'INPUT' in line]
   #  outputs = [line for line in bench if 'OUTPUT' in line]
    circuit = [line for line in bench if (
        line not in inputs) and 'OUTPUT' not in line]

    gateOut = [line.split(' = ')[0] for line in circuit]
    gate = [line.split(' = ')[1] for line in circuit]
    gate = [[line.split('(')[0]] + line.split('(')
            [1].replace(')', '').split(', ') for line in gate]

    inputs = [line.split('(')[1].replace(')', '') for line in inputs]
   #  outputs = [line.split('(')[1].replace(')', '') for line in outputs]
    circ = {gateOut[i]: line for i, line in enumerate(gate)}

    return circ, inputs


In [4]:
def sim(input_comb, circ, inputs):
   netlist = {i: -1 for i in (inputs + list(circ.keys()))}
   for i, val in enumerate(input_comb):
      netlist[inputs[i]] = val

   for wire in netlist:
      if netlist[wire] == -1:
         # visited = set()
         netlist[wire] = simDFS(circ, wire, netlist)
   return netlist


# circ, wire: G4, netlist: {G1: 0, G2: 1, G3: 0, G4: -1, G5: -1}
def simDFS(circ, wire, netlist):
   if netlist[wire] != -1:
      return netlist[wire]
   else:
      gate = circ[wire][0]
      wire_in = circ[wire][1:]
      val_in = []
      for wire in wire_in:
         val_in.append(simDFS(circ, wire, netlist))
      return gateSim(gate, val_in)


def gateSim(gate, inputs):  # gate: 'OR', inputList: [1, 0]
   if gate == 'NOT':
      return 1 - inputs[0]
   elif gate == 'AND':
      return inputs[1] and inputs[0]
   elif gate == 'NAND':
      return 1 - (inputs[1] and inputs[0])
   elif gate == 'OR':
      return inputs[1] or inputs[0]
   elif gate == 'NOR':
      return 1 - (inputs[1] or inputs[0])
   elif gate == 'XOR':
      return inputs[1] ^ inputs[0]


def getControl(benchfile):
   circ, wire_in = createDict(benchfile)
   #  df = pd.read_table(testfile, delimiter='\t')

   # generate truth table
   tt_inputs = list(product([0, 1], repeat=len(wire_in)))

   tt = {index: [-1] for index in (wire_in + list(circ.keys()))}

   for line in tt_inputs:
      netlist = sim(line, circ, wire_in)
      for key, val in tt.items():
         val.append(netlist[key])

   df = pd.DataFrame(tt).drop(0).reset_index(drop=True)
   print(df)
   # sorting truth table
   wire_in.reverse()
   for i in wire_in:
      df = df.sort_values(by=i)
   wire_in.reverse()
   print(df)

   # getting control values
   control = 0
   for input in wire_in:
      df = df.sort_values(by=input).reset_index(drop=True)
      # df = df.reset_index(drop=True)
      for key in circ:
         # print("control value of " + input + " on wire " + key + ":", end=' ')
         for index in range(len(df) // 2):
            control += df.loc[index, key] ^ df.loc[(index + len(df) // 2), key]
         # print(control)
         control = 0


In [5]:
circ, wire_in = createDict('s27.bench')


# generate truth table
tt_inputs = list(product([0, 1], repeat=len(wire_in)))

tt = {index: [-1] for index in (wire_in + list(circ.keys()))}

for line in tt_inputs:
      netlist = sim(line, circ, wire_in)
      for key, val in tt.items():
         val.append(netlist[key])

df = pd.DataFrame(tt).drop(0).reset_index(drop=True)
# print(df.to_string())


# getting control values
control_df = pd.DataFrame(np.zeros((len(circ), len(wire_in))), index=circ.keys(), columns=wire_in)
control = 0
for input in wire_in:
   df = df.sort_values(by=input).reset_index(drop=True)
   # df = df.reset_index(drop=True)
   for key in circ:
      # print("control value of " + input + " on wire " + key + ":", end=' ')
      for index in range(len(df) // 2):
         control += df.loc[index, key] ^ df.loc[(index + len(df) // 2), key]
      # print(control)
      control_df.loc[key, input] = control / (2 ** len(wire_in))
      control = 0

print(control_df.to_string())


           1         2         3         4         5         6         7
8   0.500000  0.265625  0.265625  0.171875  0.234375  0.312500  0.250000
9   0.250000  0.203125  0.187500  0.156250  0.187500  0.250000  0.203125
10  0.125000  0.250000  0.187500  0.203125  0.156250  0.171875  0.250000
11  0.250000  0.281250  0.234375  0.375000  0.328125  0.218750  0.265625
12  0.109375  0.265625  0.375000  0.296875  0.203125  0.203125  0.281250
13  0.265625  0.265625  0.250000  0.187500  0.203125  0.312500  0.296875
14  0.296875  0.234375  0.218750  0.187500  0.218750  0.312500  0.281250
15  0.156250  0.125000  0.125000  0.109375  0.171875  0.140625  0.171875
16  0.156250  0.125000  0.125000  0.109375  0.171875  0.140625  0.171875
17  0.468750  0.265625  0.265625  0.171875  0.234375  0.312500  0.265625


In [6]:
# control_df = pd.DataFrame(np.random.randn(len(circ), len(wire_in)), index=circ.keys(), columns=wire_in)

# print(control_df.to_string())